In [ ]:
!pip install llama-index docx2txt torch sentence-transformers

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

doc_folder_path = f'{root_dir}Super Doctor/Pharmbot_fulldoc'
print(os.listdir(doc_folder_path))

In [ ]:
import os

openai_api_key = 'xxx'
os.environ['OPENAI_API_KEY'] = openai_api_key

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from llama_index import SimpleDirectoryReader

reader = SimpleDirectoryReader(
    input_dir=doc_folder_path,
    recursive=True,
)

documents = []
for docs in reader.iter_data():
    for doc in docs:
        # do something with the doc
        doc.text = doc.text.upper()
        documents.append(doc)

In [ ]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

99 

<class 'llama_index.schema.Document'>
Doc ID: e2e9d980-8c30-409b-9d64-8a904d190d9c
Text: SINGHEALTH NAME   ACETAZOLAMIDE    ACETAZOLAMIDE ADVERSE
REACTIONS  THE FOLLOWING ADVERSE DRUG REACTIONS AND INCIDENCES ARE
DERIVED FROM PRODUCT LABELING UNLESS OTHERWISE SPECIFIED.  FREQUENCY
NOT DEFINED:  CARDIOVASCULAR: FLUSHING  DERMATOLOGIC: ALLERGIC SKIN
REACTION, SKIN PHOTOSENSITIVITY  ENDOCRINE & METABOLIC: ELECTROLYTE
DISORDER, GLYCOSUR...


In [ ]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [ ]:
from llama_index.node_parser import HierarchicalNodeParser

# create the hierarchical node parser w/ default settings
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)

In [ ]:
nodes = node_parser.get_nodes_from_documents([document])

In [ ]:
from llama_index.node_parser import get_leaf_nodes

leaf_nodes = get_leaf_nodes(nodes)
print(leaf_nodes[30].text)

NOTE: KIDNEY FUNCTION-ADJUSTED DOSE RECOMMENDATIONS ARE BASED ON USUAL DOSES UP TO 1 G/DAY. THERE ARE NO DOSAGE ADJUSTMENTS NECESSARY FOR ANY DEGREE OF KIDNEY DYSFUNCTION FOR SINGLE DOSES OF 500 MG (IE, ELEVATED INTRAOCULAR PRESSURE OR INITIAL TREATMENT OF METABOLIC ALKALOSIS) (REF).


In [ ]:
nodes_by_id = {node.node_id: node for node in nodes}

parent_node = nodes_by_id[leaf_nodes[30].parent_node.node_id]
print(parent_node.text)

ORAL (IMMEDIATE RELEASE), IV: 500 MG ONCE (REF).

IDIOPATHIC INTRACRANIAL HYPERTENSION (OFF-LABEL USE):

ORAL (IMMEDIATE RELEASE, EXTENDED RELEASE): INITIAL: 250 TO 500 MG TWICE DAILY; INCREASE AS TOLERATED BY 250 MG EVERY WEEK TO REACH DESIRED CLINICAL EFFECT OR A MAXIMUM OF 4 G/DAY (REF).

METABOLIC ALKALOSIS (OFF-LABEL USE):

NOTE: IN GENERAL, USE AFTER TREATMENT OF UNDERLYING CAUSES AND REPLACEMENT OF ISOTONIC SALINE OR POTASSIUM CHLORIDE WHEN APPROPRIATE, OR WHEN ADDITIONAL FLUIDS ARE CONTRAINDICATED (REF).

IV (PREFERRED), ORAL (IMMEDIATE RELEASE): 500 MG AS A SINGLE DOSE; MAY REPEAT AS NEEDED BASED UPON ACID-BASE STATUS WITH 250 TO 500 MG UP TO ONCE OR TWICE DAILY (REF).

DOSAGE ADJUSTMENT FOR CONCOMITANT THERAPY: SIGNIFICANT DRUG INTERACTIONS EXIST, REQUIRING DOSE/FREQUENCY ADJUSTMENT OR AVOIDANCE. CONSULT DRUG INTERACTIONS DATABASE FOR MORE INFORMATION.

ACETAZOLAMIDE DOSING: KIDNEY IMPAIRMENT: ADULT

THE RENAL DOSING RECOMMENDATIONS ARE BASED UPON THE BEST AVAILABLE EVIDENCE 

In [ ]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-4", temperature=0.1)

In [ ]:
from llama_index import ServiceContext

auto_merging_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    node_parser=node_parser,
)

In [ ]:
from llama_index import VectorStoreIndex, StorageContext

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

automerging_index = VectorStoreIndex(
    leaf_nodes, storage_context=storage_context, service_context=auto_merging_context
)

automerging_index.storage_context.persist(persist_dir="./merging_index")

In [ ]:
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index.retrievers import AutoMergingRetriever
from llama_index.query_engine import RetrieverQueryEngine

automerging_retriever = automerging_index.as_retriever(
    similarity_top_k=20
)

retriever = AutoMergingRetriever(
    automerging_retriever,
    automerging_index.storage_context,
    verbose=True
)

rerank = SentenceTransformerRerank(top_n=10, model="BAAI/bge-reranker-base")

auto_merging_engine = RetrieverQueryEngine.from_args(
    automerging_retriever, node_postprocessors=[rerank]
)

In [ ]:
medications = ['Enoxaparin', 'Actrapid', 'Lantus', 'Novorapid', 'Aspirin', 'Clopidogrel', 'Omeprazole', 'Glyceryl Trinitrate', 'Linagliptin', 'Bisoprolol', 'Perindopril', 'Neurobion']

In [ ]:
medications_indication_task = [medication + " " + "Use: Labeled Indications, Use: Off-Label: Adult" for medication in medications]

medications_indication_task_str = '"' +'""'.join(medications_indication_task)+ '"'

medications_dosing_task = [medication + " " + "CrCl, Dosing: Adult, Dosing: Altered Kidney Function: Adult, Dosing: Hepatic Impairment: Adult" for medication in medications]

medications_dosing_task_str = '"' +'""'.join(medications_dosing_task)+ '"'

medications_adr_task = [medication + " " + "Adverse Reactions (Significant): Considerations, Adverse Reactions" for medication in medications]

medications_adr_task_str = '"' +'""'.join(medications_adr_task)+ '"'

medications_ddi_task = [medication + " " + "Drug Interactions" for medication in medications]

medications_ddi_task_str = '"' +'""'.join(medications_ddi_task)+ '"'

medications_moa_task = [medication + " " + "Mechanism of Action, Pharmacokinetics (Adult Data Unless Noted)" for medication in medications]

medications_moa_task_str = '"' +'""'.join(medications_moa_task)+ '"'

medications_indication = medications_indication_task_str.split('"')
medications_indication = [med.strip() for med in medications_indication if med.strip()]
medications_dosing = medications_dosing_task_str.split('"')
medications_dosing = [med.strip() for med in medications_dosing if med.strip()]
medications_adr = medications_adr_task_str.split('"')
medications_adr = [med.strip() for med in medications_adr if med.strip()]
medications_ddi = medications_ddi_task_str.split('"')
medications_ddi = [med.strip() for med in medications_ddi if med.strip()]
medications_moa = medications_moa_task_str.split('"')
medications_moa = [med.strip() for med in medications_moa if med.strip()]

indication_responses = [str(auto_merging_engine.query(med)) for med in medications_indication]
indication_combined_responses = "\n".join(indication_responses)

dosing_responses = [str(auto_merging_engine.query(med)) for med in medications_dosing]
dosing_combined_responses = "\n".join(dosing_responses)

adr_responses = [str(auto_merging_engine.query(med)) for med in medications_adr]
adr_combined_responses = "\n".join(adr_responses)

ddi_responses = [str(auto_merging_engine.query(med)) for med in medications_ddi]
ddi_combined_responses = "\n".join(ddi_responses)

moa_responses = [str(auto_merging_engine.query(med)) for med in medications_moa]
moa_combined_responses = "\n".join(moa_responses)

In [ ]:
medications_indication_task = [medication + " " + "Use: Labeled Indications, Use: Off-Label: Adult" for medication in medications]

medications_indication_task_str = '"' +'""'.join(medications_indication_task)+ '"'

medications_dosing_task = [medication + " " + "CrCl, Dosing: Adult, Dosing: Altered Kidney Function: Adult, Dosing: Hepatic Impairment: Adult" for medication in medications]

medications_dosing_task_str = '"' +'""'.join(medications_dosing_task)+ '"'

medications_adr_task = [medication + " " + "Adverse Reactions (Significant): Considerations, Adverse Reactions" for medication in medications]

medications_adr_task_str = '"' +'""'.join(medications_adr_task)+ '"'

medications_ddi_task = [medication + " " + "Drug Interactions" for medication in medications]

medications_ddi_task_str = '"' +'""'.join(medications_ddi_task)+ '"'

medications_moa_task = [medication + " " + "Mechanism of Action, Pharmacokinetics (Adult Data Unless Noted)" for medication in medications]

medications_moa_task_str = '"' +'""'.join(medications_moa_task)+ '"'

In [ ]:
medications_moa_task_str

'"Enoxaparin Mechanism of Action, Pharmacokinetics (Adult Data Unless Noted)""Actrapid Mechanism of Action, Pharmacokinetics (Adult Data Unless Noted)""Lantus Mechanism of Action, Pharmacokinetics (Adult Data Unless Noted)""Novorapid Mechanism of Action, Pharmacokinetics (Adult Data Unless Noted)""Aspirin Mechanism of Action, Pharmacokinetics (Adult Data Unless Noted)""Clopidogrel Mechanism of Action, Pharmacokinetics (Adult Data Unless Noted)""Omeprazole Mechanism of Action, Pharmacokinetics (Adult Data Unless Noted)""Glyceryl Trinitrate Mechanism of Action, Pharmacokinetics (Adult Data Unless Noted)""Linagliptin Mechanism of Action, Pharmacokinetics (Adult Data Unless Noted)""Bisoprolol Mechanism of Action, Pharmacokinetics (Adult Data Unless Noted)""Perindopril Mechanism of Action, Pharmacokinetics (Adult Data Unless Noted)""Neurobion Mechanism of Action, Pharmacokinetics (Adult Data Unless Noted)"'

In [ ]:
indication_combined_responses

dosing_combined_responses

adr_combined_responses

ddi_combined_responses

moa_combined_responses

"The mechanism of action of enoxaparin involves the inhibition of protein synthesis and cell death in susceptible organisms by interacting with DNA and causing a loss of helical DNA structure and strand breakage. In terms of pharmacokinetics, enoxaparin is well absorbed when taken orally and has a distribution volume of approximately 34 L. It has a higher ratio of anti-factor Xa to anti-factor IIA activity compared to unfractionated heparin. Additionally, the average molecular weight of enoxaparin is 4500 Daltons, with different components distributed across various molecular weight ranges.\nActrapid acts by binding to specific receptors on target tissues to regulate the metabolism of carbohydrates, proteins, and fats. The target organs for Actrapid include the liver, skeletal muscle, and adipose tissue. As for its pharmacokinetics, the onset of action of Actrapid varies depending on the route of administration, with deeper intramuscular injections resulting in more rapid absorption an

In [ ]:
moa_combined_responses

"The mechanism of action of enoxaparin involves the inhibition of protein synthesis and cell death in susceptible organisms by interacting with DNA and causing a loss of helical DNA structure and strand breakage. In terms of pharmacokinetics, enoxaparin is well absorbed when taken orally and has a distribution volume of approximately 34 L. It has a higher ratio of anti-factor Xa to anti-factor IIA activity compared to unfractionated heparin. Additionally, the average molecular weight of enoxaparin is 4500 Daltons, with different components distributed across various molecular weight ranges.\nActrapid acts by binding to specific receptors on target tissues to regulate the metabolism of carbohydrates, proteins, and fats. The target organs for Actrapid include the liver, skeletal muscle, and adipose tissue. As for its pharmacokinetics, the onset of action of Actrapid varies depending on the route of administration, with deeper intramuscular injections resulting in more rapid absorption an

In [ ]:
medications_moa = medications_moa_task_str.split('"')
medications_moa = [med.strip() for med in medications_moa if med.strip()]

moa_responses = [str(auto_merging_engine.query(med)) for med in medications_moa]
moa_combined_responses = "\n".join(moa_responses)

moa_combined_responses

"The mechanism of action of enoxaparin involves the inhibition of protein synthesis and cell death in susceptible organisms by interacting with DNA and causing a loss of helical DNA structure and strand breakage. In terms of pharmacokinetics, enoxaparin is well absorbed when taken orally and has a distribution volume of approximately 34 L. It has a higher ratio of anti-factor Xa to anti-factor IIA activity compared to unfractionated heparin. The average molecular weight of enoxaparin is 4500 Daltons, and it is distributed in different molecular weight ranges. The onset of action of enoxaparin is around 3 to 5 hours, and its duration of action is approximately 12 hours.\nActrapid acts by binding to specific receptors on target tissues to regulate the metabolism of carbohydrates, proteins, and fats. The target organs for Actrapid include the liver, skeletal muscle, and adipose tissue. As for its pharmacokinetics, the onset of action of Actrapid varies depending on the route of administra

In [ ]:
auto_merging_response = auto_merging_engine.query(
    "Enoxaparin Mechanism of Action, Pharmacokinetics (Adult Data Unless Noted)"
)

In [ ]:
str(auto_merging_response)

'Enoxaparin is a low molecular weight heparin that consists of components with molecular weights ranging from 4000 to 30,000 Daltons. Its mechanism of action involves interacting with antithrombin and factor Xa to inhibit protein synthesis and cell death in susceptible organisms. In terms of pharmacokinetics, enoxaparin is well absorbed when taken orally and has a distribution volume of approximately 34 L. It has a higher ratio of anti-factor Xa to anti-factor IIA activity compared to unfractionated heparin. The onset of action is typically 3 to 5 hours, and the duration of action for a 40 mg dose is approximately 12 hours. The pharmacokinetics of enoxaparin may be altered in patients with severe kidney impairment, leading to an increased area under the curve (AUC).'

In [ ]:
from llama_index.response.notebook_utils import display_response

display_response(auto_merging_response)

**`Final Response:`** The dosing of enoxaparin may need to be altered in patients with kidney impairment. This recommendation is based on the best available evidence and clinical expertise. It is important to note that serious bleeding complications have been reported in patients with severe kidney failure or who are dialysis dependent. However, some retrospective studies suggest that enoxaparin may be used for deep vein thrombosis prophylaxis in dialysis patients without an increased risk of bleeding. It is also important to consider the increased risk of bleeding when repeated dosing of enoxaparin is required for spinal/epidural anesthesia.